# Experiment Notebook: Network Issuance and Inflation Rate

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis: Inflation Rate and ETH Supply Over Time](#Analysis:-Inflation-Rate-and-ETH-Supply-Over-Time)

# Experiment Summary 

The purpose of this notebook is to explore the ETH issuance and resulting annualized inflation rate across different time horizons, adoption scenarios, and network upgrade stages.

# Experiment Assumptions

See [assumptions document](ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
import setup

import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from model.types import Stage
from data.historical_values import df_ether_supply

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.templates.time_domain_analysis as time_domain_analysis

In [ ]:
# Fetch the time-domain analysis experiment
experiment = time_domain_analysis.experiment
# Create a copy of the experiment simulation
simulation = copy.deepcopy(experiment.simulations[0])

In [ ]:
# Experiment configuration

simulation_names = {
    'Validator Adoption Scenarios': [
        'Normal Adoption',
        'Low Adoption',
        'High Adoption'
    ],
    'PoS Launch Date Scenarios': [
        "2021/12/1",
        "2022/12/1",
        "2023/12/1",
        "2024/12/1",
    ],
    'EIP1559 Scenarios': [
        'Disabled',
        'Enabled: Steady State',
        'Enabled: MEV',
    ]
}

simulation_1 = copy.deepcopy(simulation)
simulation_1.model.params.update({
    'validator_process': [
        lambda _run, _timestep: 3,  # Normal adoption: current average active validators per epoch from Beaconscan
        lambda _run, _timestep: 3 * 0.5,  # Low adoption: 50% lower scenario
        lambda _run, _timestep: 3 * 1.5,  # High adoption: 50% higher scenario
    ],
})

simulation_2 = copy.deepcopy(simulation)
simulation_2.model.params.update({
    'date_pos': [
        datetime.strptime("2021/12/1", "%Y/%m/%d"),
        datetime.strptime("2022/03/1", "%Y/%m/%d"),
        datetime.strptime("2022/06/1", "%Y/%m/%d"),
        datetime.strptime("2022/09/1", "%Y/%m/%d"),
    ],
})

simulation_3 = copy.deepcopy(simulation)
simulation_3.model.params.update({
    'eip1559_basefee_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 100, # Enabled: Steady state
        lambda _run, _timestep: 70, # Enabled: MEV
    ],  # Gwei per gas
    'eip1559_tip_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 1, # Enabled: Steady state
        lambda _run, _timestep: 30, # Enabled: MEV
    ],  # Gwei per gas
})

experiment.simulations = [
    simulation_1,
    simulation_2,
    simulation_3
]

In [ ]:
df, _exceptions = run(experiment)

# Analysis: Inflation Rate and ETH Supply Over Time

TODO: update description re. scenarios when finalized

This analysis allows the exploration of inflation rate and ETH supply over time, and supports the three adoption scenarios introduced in the second analysis notebook. The activation dates of the two major milestones in this analysis, EIP1559 and Proof of Stake, can be customized to simulate bespoke scenarios.

## WIP Visualisations

TODO: combine the following two visualizations, or create sliders for each parameter option

See WIP combination `plot_eth_supply_and_inflation_over_all_stages_wip(...)`

In [ ]:
visualizations.plot_network_issuance_scenarios(df, simulation_names)

In [ ]:
from radcad.utils import generate_cartesian_product_parameter_sweep

simulation_0 = copy.deepcopy(simulation)

param_sweep = generate_cartesian_product_parameter_sweep({
#     'validator_process': [
#         lambda _run, _timestep: 3,  # Normal adoption: current average active validators per epoch from Beaconscan
#         lambda _run, _timestep: 3 * 0.5,  # Low adoption: 50% lower scenario
#         lambda _run, _timestep: 3 * 1.5,  # High adoption: 50% higher scenario
#     ],
#     'date_pos': [
#         datetime.strptime("2021/12/1", "%Y/%m/%d"),
#         datetime.strptime("2022/12/1", "%Y/%m/%d"),
#         datetime.strptime("2023/12/1", "%Y/%m/%d"),
#         datetime.strptime("2024/12/1", "%Y/%m/%d"),
#     ],
    'eip1559_basefee_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 100, # Enabled: Steady state
        lambda _run, _timestep: 70, # Enabled: MEV
    ],  # Gwei per gas
    'eip1559_tip_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 1, # Enabled: Steady state
        lambda _run, _timestep: 30, # Enabled: MEV
    ],  # Gwei per gas
})

simulation_0.model.params.update(param_sweep)

In [ ]:
df_0, _exceptions = run(simulation_0)

In [ ]:
# TODO: Create a dropdown to choose between the three adoption scenarios
# TODO: Create a dropdown or slider to allow for the customization of EIP1559 and PoS dates
visualizations.plot_eth_supply_and_inflation_over_all_stages(df_ether_supply, df_0)

In [ ]:
simulation_dash = copy.deepcopy(simulation)

def run_simulation(validators_per_epoch, pos_launch_date, eip1559_basefee, eip1559_avg_tip):
    simulation_dash.model.params.update({
        'validator_process': [
            lambda _run, _timestep: float(validators_per_epoch),
        ],
        'date_pos': [
            datetime.strptime(pos_launch_date, "%Y/%m/%d")
        ],
        'eip1559_basefee_process': [
            lambda _run, _timestep: float(eip1559_basefee), 
        ],  # Gwei per gas
        'eip1559_tip_process': [
            lambda _run, _timestep: float(eip1559_avg_tip),
        ],  # Gwei per gas
    })


    df, _exceptions = run(simulation_dash)
    
    return df, simulation_dash.model.params

In [ ]:
import plotly.express as px
from datetime import date
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly_theme


# Load Data
df = px.data.tips()
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.Div([
        html.Label([
            "Validator Adoption Scenario",
            dcc.Dropdown(
                id='validator-dropdown', clearable=False,
                value='3', options=[
                    {'label': 'Normal Adoption', 'value': 3},
                    {'label': 'Low Adoption', 'value': 3 * 0.5},
                    {'label': 'High Adoption', 'value': 3 * 1.5},
                    {'label': 'Custom', 'value': 'Custom'}
                ]),
            html.Label([
                "Validator Adoption",
                dcc.Slider(
                    id='validator-adoption-slider',
                    min=0,
                    max=10,
                    step=0.5,
                    marks={
                        0: '0',
                        5: '5',
                        10: '10',
                    },
                    value=3,
                    tooltip={'placement': 'top'}
                )]),
        ], style={'float': 'left', 'width': '300px', 'height': '50px'}),
        html.Label([
            "PoS Launch Date",
            dcc.Dropdown(
                id='pos-launch-date-dropdown', clearable=False,
                value='2021/12/1', options=[
                    {'label': 'Dec 2021', 'value': '2021/12/1'},
                    {'label': 'Mar 2022', 'value': '2022/03/1'},
                    {'label': 'Jun 2022', 'value': '2022/06/1'},
                    {'label': 'Sep 2022', 'value': '2022/09/1'}
                ]),
        ], style={'float': 'left', 'width': '300px', 'height': '50px'}),
        html.Label([
            "EIP1559 Scenarios",
            dcc.Dropdown(
                id='eip1559-dropdown', clearable=False,
                value='Disabled', options=[
                    {'label': 'Disabled', 'value': 'Disabled'},
                    {'label': 'Enabled: Steady State', 'value': 'Enabled: Steady State'},
                    {'label': 'Enabled: MEV', 'value': 'Enabled: MEV'},
                    {'label': 'Custom', 'value': 'Custom'},
                ]),
            html.Label([
                "EIP1559 Base Fee",
                dcc.Slider(
                    id='eip1559-basefee-slider',
                    min=0,
                    max=100,
                    step=5,
                    marks={
                        0: '0',
                        25: '25',
                        50: '50',
                        75: '75',
                        100: '100'
                    },
                    value=0,
                    tooltip={'placement': 'top'}
                )]),
            html.Label([
                "EIP1559 Validator Tips",
                dcc.Slider(
                    id='eip1559-validator-tips-slider',
                    min=0,
                    max=100,
                    step=5,
                    marks={
                        0: '0',
                        25: '25',
                        50: '50',
                        75: '75',
                        100: '100'
                    },
                    value=0,
                    tooltip={'placement': 'top'}
                )])
        ], style={'float': 'left', 'width': '300px', 'height': '90px'}),
    ], style={'width': '900px', 'height': '200px'}),
    html.Div([
        dcc.Loading(
            id="loading-1",
            children=[dcc.Graph(id="graph")],
            type="default"
        )
    ]),
])


# Callbacks
@app.callback(
    Output('eip1559-basefee-slider', 'value'),
    Output('eip1559-validator-tips-slider', 'value'),
    [Input('eip1559-dropdown', 'value')]
)
def update_eip1559_sliders_by_scenarios(eip1559_dropdown):
    eip1559_scenarios = {'Disabled': [0, 0], 'Enabled: Steady State': [100, 1], 'Enabled: MEV': [70, 30]}
    if eip1559_dropdown == 'Custom':
        raise PreventUpdate
    return eip1559_scenarios[eip1559_dropdown][0], eip1559_scenarios[eip1559_dropdown][1]


@app.callback(
    Output('validator-adoption-slider', 'value'),
    [Input('validator-dropdown', 'value')]
)
def update_validator_adoption_sliders_by_scenarios(validator_dropdown):
    return validator_dropdown


# Define callback to update graph
@app.callback(
    Output('validator-dropdown', 'value'),
    Output('eip1559-dropdown', 'value'),
    Output('graph', 'figure'),
    [Input("validator-adoption-slider", "value"),
     Input("pos-launch-date-dropdown", "value"),
     Input("eip1559-basefee-slider", "value"),
     Input("eip1559-validator-tips-slider", "value")]
)
def update_output_graph(validator_adoption, pos_launch_date, eip1559_basefee, eip1559_validator_tips):
    df_0, parameters = run_simulation(validator_adoption, pos_launch_date, eip1559_basefee, eip1559_validator_tips)
    
    if validator_adoption not in [1.5, 3, 4.5]:
        validator_adoption = 'Custom'

    eip1559_fees = [eip1559_basefee, eip1559_validator_tips]
    if eip1559_fees in [[0, 0], [100, 1], [70, 30]]:
        if eip1559_fees == [0, 0]:
            eip1559 = 'Disabled'
        elif eip1559_fees == [100, 1]:
            eip1559 = 'Enabled: Steady State'
        else:
            eip1559 = 'Enabled: MEV'
    else:
        eip1559 = 'Custom'

    return (
        validator_adoption,
        eip1559,
        visualizations.plot_eth_supply_and_inflation_over_all_stages(df_ether_supply, df_0, parameters=parameters)
    )


# Run app and display result inline in the notebook
app.run_server(mode='inline')